## biGRU-attention model for facenet-embedded-data

Needs output data from <b>01-7 process-data-facenet</b>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GRU, Attention, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as kr
import pickle
from data_split import train_valid_test_split

### Load data

In [ ]:
####
#parameters
test_rate = 0.15
valid_rate = 0.15
window = 30 #frame window
seed = 123456

####
#load focus videos
of = open('facenetdata.obj', 'rb')
data, labels = pickle.load(of)
of.close()

####
#transform classes
from sklearn.preprocessing import OrdinalEncoder
label_nums = OrdinalEncoder().fit_transform(labels)

####
#split into train/valid/test
X_train,X_valid,X_test,y_train,y_valid,y_test = train_valid_test_split(data,label_nums,test_rate,valid_rate,window,seed)

### Model building

In [ ]:
####
#parameters
gru_size = 512
out_dense = [512]
dropout = False
droprate = 0.4

####
#functional codes
face_in = tf.keras.Input(shape=(window,X_train_face.shape[-1]))
# forward direction
x = face_in
x1 = GRU(gru_size, return_sequences=True)(x)
x1 = BatchNormalization()(x1)
# backward direction
x2 = tf.reverse(x, axis=[1])
x2 = GRU(gru_size, return_sequences=True)(x2)
x2 = BatchNormalization()(x2)
# combine
x = tf.concat([x1,x2], axis=-1)
if dropout:
    x = Dropout(droprate)(x)
# Self-Attention Block
x = Attention()([x, x])
x = tf.keras.layers.GlobalMaxPooling1D()(x)
# Output Block
for d in out_dense:
    x = Dense(d, activation="relu")(x)
    if dropout:
        x = Dropout(droprate)(x)
output = Dense(units=1, activation='softmax')(x)
# Build and Compile model
model = Model(inputs=[face_in], outputs=[output])
model.summary()

### Training

In [ ]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=[X_valid, y_valid], batch_size=512, epochs=1000)

### Testing

In [ ]:
model.evaluate(X_test, y_test)